In [ ]:
from google.colab import drive

# Conecta o Colab ao Google Drive
drive.mount('/content/drive')


In [ ]:
!pip install xarray rioxarray netCDF4 matplotlib


In [ ]:
# ----------------- CÉLULA 3 -----------------
# Definindo os caminhos para os arquivos no Google Drive
path_pace = '/content/drive/My Drive/projeto_tubaroes/PACE_OCI.clorofila_fitoplancton.nc'
path_modis = '/content/drive/My Drive/projeto_tubaroes/AQUA_MODIS_Temperature.nc'
path_swot = '/content/drive/My Drive/projeto_tubaroes/SWOT_WindWave.nc'

# Imprimindo para confirmar que os caminhos estão corretos
print("Arquivo PACE:", path_pace)
print("Arquivo MODIS:", path_modis)
print("Arquivo SWOT:", path_swot)


In [ ]:
# ---------------- CÉLULA 4 ----------------
# Abrindo os arquivos netCDF com caminhos corretos

import xarray as xr
import os

# Caminhos corretos dos arquivos no Google Drive
path_pace = "/content/drive/My Drive/projetos_tubaroes/PACE_OCI.clorofila_fitoplancton.nc"
path_modis = "/content/drive/My Drive/projetos_tubaroes/AQUA_MODIS_Temperature.nc"
path_swot = "/content/drive/My Drive/projetos_tubaroes/SWOT_WindWave.nc"

# Função para abrir dataset de forma segura
def abrir_dataset(path):
    if os.path.exists(path):
        ds = xr.open_dataset(path)
        print(f"Arquivo carregado com sucesso: {os.path.basename(path)}")
        return ds
    else:
        print(f"Erro: Arquivo não encontrado em {path}")
        return None

# Abrindo os arquivos
ds_pace = abrir_dataset(path_pace)
ds_modis = abrir_dataset(path_modis)
ds_swot = abrir_dataset(path_swot)

# Exibir informações básicas de cada dataset
if ds_pace is not None:
    print("\n--- Dados PACE (Clorofila-a) ---")
    print(ds_pace)

if ds_modis is not None:
    print("\n--- Dados MODIS (Temperatura do mar) ---")
    print(ds_modis)

if ds_swot is not None:
    print("\n--- Dados SWOT (Altura da superfície do mar e ondas) ---")
    print(ds_swot)


In [ ]:
!pip install cartopy
import cartopy

In [ ]:
# -------- Célula 5 corrigida com Cartopy --------
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def plot_global_map(data, lats, lons, title, cmap='viridis', vmin=None, vmax=None):
    """
    Plota dados globais sobre mapa usando Cartopy
    """
    plt.figure(figsize=(14,7))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Contornos da Terra
    ax.add_feature(cfeature.LAND, zorder=0, edgecolor='black')
    ax.add_feature(cfeature.OCEAN, zorder=0)
    ax.add_feature(cfeature.COASTLINE)
    ax.gridlines(draw_labels=True)

    # Transformar meshgrid em 2D para pcolormesh
    mesh = ax.pcolormesh(lons, lats, data, cmap=cmap, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    plt.colorbar(mesh, ax=ax, orientation='vertical', label=title)
    plt.title(title)
    plt.show()

# -------- PACE – Clorofila-a --------
chlor_a = ds_pace['chlor_a'].values
lats_pace = ds_pace['lat'].values
lons_pace = ds_pace['lon'].values
lon2d_pace, lat2d_pace = np.meshgrid(lons_pace, lats_pace)

plot_global_map(chlor_a, lat2d_pace, lon2d_pace, 'Clorofila-a (mg/m³) – PACE', cmap='Greens')

# -------- MODIS – Temperatura do mar --------
sst = ds_modis['sst'].values
lats_modis = ds_modis['lat'].values
lons_modis = ds_modis['lon'].values
lon2d_modis, lat2d_modis = np.meshgrid(lons_modis, lats_modis)

plot_global_map(sst, lat2d_modis, lon2d_modis, 'Temperatura da Superfície do Mar (°C) – MODIS', cmap='RdYlBu_r', vmin=0, vmax=35)


# -------- SWOT – Altura das Ondas (scatter para dados irregulares) --------
swh = ds_swot['swh_karin'].values  # Pode ser (num_lines, num_pixels) ou (num_lines, num_sides, num_pixels)
# Se ainda for 3D, tira média sobre 'num_sides'
if swh.ndim == 3:
    swh = np.mean(swh, axis=1)  # agora swh é (num_lines, num_pixels)

# Achata para 1D (todos os pontos)
swh_flat = swh.flatten()
lats_flat = ds_swot['latitude'].values
lons_flat = ds_swot['longitude'].values
# Se tiver 3D, achata também
if lats_flat.ndim == 3:
    lats_flat = np.mean(lats_flat, axis=1).flatten()
    lons_flat = np.mean(lons_flat, axis=1).flatten()
else:
    lats_flat = lats_flat.flatten()
    lons_flat = lons_flat.flatten()

# Plot usando scatter
plt.figure(figsize=(14,7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.OCEAN)
ax.gridlines(draw_labels=True)

sc = ax.scatter(lons_flat, lats_flat, c=swh_flat, s=1, cmap='Blues', vmin=0, vmax=10, transform=ccrs.PlateCarree())
plt.colorbar(sc, ax=ax, orientation='vertical', label='Altura das Ondas (m) – SWOT')
plt.title('Altura das Ondas (m) – SWOT')
plt.show()


In [ ]:
# ----------------- CÉLULA 6 -----------------
# Pré-processamento: regridding, rasterização do SWOT e normalização
import numpy as np
import xarray as xr
import os

# Caminho para salvar o índice final no Drive
saida_nc = "/content/drive/My Drive/projetos_tubaroes/indice_tubarao.nc"

print("INICIANDO PRÉ-PROCESSAMENTO...")

# 1) Definir grade-alvo (usaremos a grade do PACE - 0.1° já carregada)
lat_target = np.array(ds_pace['lat'].values)    # pode estar decrescente
lon_target = np.array(ds_pace['lon'].values)

# Ordenar centros em ordem crescente (necessário para histogram bins)
lat_centers = np.sort(lat_target)
lon_centers = np.sort(lon_target)

# passos aproximados (assumimos grade regular)
dlat = np.mean(np.diff(lat_centers))
dlon = np.mean(np.diff(lon_centers))
print(f"Grade alvo: {len(lat_centers)} lat x {len(lon_centers)} lon  |  dlat={dlat:.4f}, dlon={dlon:.4f}")

# 2) Regriddar MODIS (SST) para a grade do PACE usando xarray.interp (mais simples)
print("Reamostrando MODIS (SST) para a grade do PACE...")
try:
    # alguns datasets têm lat/lon como coordenadas nomeadas 'lat' e 'lon' (verifique)
    sst_interp = ds_modis['sst'].interp(lat=lat_centers, lon=lon_centers, method='nearest')
    sst_grid = sst_interp.values  # shape (nlat, nlon)
    print("OK: reamostragem MODIS concluída com xarray.interp.")
except Exception as e:
    print("AVISO: reamostragem com xarray.interp falhou:", e)
    print("Tentando interpolação simples por nearest usando numpy (fallback)...")
    # fallback: nearest neighbour manual (pode ser lento); aqui só para robustez
    # Criar meshgrid de coords MODIS e grade alvo e usar nearest via broadcasting (pesado)
    # Para simplicidade: preencher com NaN e abortar (o usuário pode instalar xesmf para regridding real)
    sst_grid = np.full((len(lat_centers), len(lon_centers)), np.nan)
    print("Fallback criou grade vazia (recomenda-se instalar xesmf para regridding mais robusto).")

# 3) Pegando clorofila (já está na grade do PACE) — garantir alinhamento com lat_centers/ lon_centers
print("Pegando clorofila (PACE) na grade alvo...")
chl = ds_pace['chlor_a'].values
# Se os lat/lon do PACE estavam decrescentes, rearranjamos para lat_centers crescentes
if not np.all(np.diff(ds_pace['lat'].values) > 0):
    # ds_pace lat estava decrescente — reorder
    order_lat = np.argsort(ds_pace['lat'].values)
    chl = chl[order_lat, :]
    print("Reordenado eixo latitude do PACE para ordem crescente.")

# 4) Rasterizar os pontos SWOT (swh_karin) para a grade alvo usando bincount / histogram2d
print("Rasterizando SWOT (swh) na grade alvo (média por célula)...")
# Extrair swh e coords
swh = ds_swot['swh_karin'].values  # possivelmente 3D (num_lines, num_sides, num_pixels) ou 2D
lat_swot = ds_swot['latitude'].values
lon_swot = ds_swot['longitude'].values

# Se for 3D, reduzir o eixo 'num_sides' se existir
if swh.ndim == 3:
    swh = np.nanmean(swh, axis=1)       # agora (num_lines, num_pixels)
    lat_swot = np.mean(lat_swot, axis=1)   # careful: may produce 2D
    lon_swot = np.mean(lon_swot, axis=1)

# Garantir arrays 2D -> achatar
swh_flat = swh.flatten()
lat_flat = lat_swot.flatten()
lon_flat = lon_swot.flatten()

# Filtrar NaNs e valores inválidos
mask_valid = (~np.isnan(swh_flat)) & (~np.isnan(lat_flat)) & (~np.isnan(lon_flat))
swh_flat = swh_flat[mask_valid]
lat_flat = lat_flat[mask_valid]
lon_flat = lon_flat[mask_valid]
print(f"Pontos SWOT válidos: {swh_flat.size}")

# Definir bin edges (centers -> edges)
lat_edges = np.linspace(lat_centers[0] - dlat/2, lat_centers[-1] + dlat/2, len(lat_centers) + 1)
lon_edges = np.linspace(lon_centers[0] - dlon/2, lon_centers[-1] + dlon/2, len(lon_centers) + 1)

# histogram2d usa (x=lat, y=lon) -> retorna array shape (nlat, nlon)
sum_hist, _, _ = np.histogram2d(lat_flat, lon_flat, bins=[lat_edges, lon_edges], weights=swh_flat)
count_hist, _, _ = np.histogram2d(lat_flat, lon_flat, bins=[lat_edges, lon_edges])

# média por célula (evitar divisão por zero)
with np.errstate(invalid='ignore', divide='ignore'):
    swh_grid = sum_hist / count_hist
swh_grid[count_hist == 0] = np.nan
print("Rasterização SWOT concluída.")

# 5) Agora temos 3 grids (nlat x nlon):
#    - chl (from PACE) already aligned (chl)
#    - sst_grid (from MODIS regridded) maybe contains NaNs if regrid failed
#    - swh_grid (from SWOT rasterization)

# Se sst_grid estiver todo NaN (fallback), substituir por valor neutro (safeguarda)
if np.all(np.isnan(sst_grid)):
    print("SST reamostrado inválido — preenchendo com NaN (atenção). Recomenda-se regridding com xesmf.")
else:
    print("SST reamostrado presente.")

# 6) Normalização / transformação das variáveis
print("Normalizando e convertendo variáveis para escala 0-1...")

# Função min-max robusta
def minmax_norm(arr):
    arr = np.array(arr, dtype=float)
    mask = ~np.isnan(arr)
    if np.any(mask):
        amin = np.nanmin(arr)
        amax = np.nanmax(arr)
        if amax - amin == 0:
            return np.zeros_like(arr)
        out = (arr - amin) / (amax - amin)
        out[~mask] = np.nan
        return out
    else:
        return arr * np.nan

chl_norm = minmax_norm(chl)
swh_norm = minmax_norm(swh_grid)

# Para SST, vamos usar uma função que dá score por proximidade a uma temperatura preferida (gaussiana)
# (sugestão: ajuste preferred_temp e sigma conforme a espécie de tubarão)
preferred_temp = 20.0   # °C (valor inicial sugerido)
sigma_temp = 5.0        # spread em °C
if np.all(np.isnan(sst_grid)):
    sst_score = np.full_like(chl_norm, np.nan)
else:
    # aplicar gaussian similarity e normalizar em seguida
    sst_score_raw = np.exp(-0.5 * ((sst_grid - preferred_temp) / sigma_temp)**2)
    sst_score = minmax_norm(sst_score_raw)

# 7) Combinar índices com pesos (α, β, γ)
alpha, beta, gamma = 0.5, 0.3, 0.2   # pesos iniciais (sugestão: ajustar via validação)
print(f"Pesos usados: alpha={alpha}, beta={beta}, gamma={gamma}")

# Combinação com atenção a NaNs: média ponderada ignorando NaNs proporcionalmente
# Calculamos soma ponderada e divisores (somatório de pesos presentes)
weights_sum = np.zeros_like(chl_norm, dtype=float)
weighted_sum = np.zeros_like(chl_norm, dtype=float)

# chl contribution
mask = ~np.isnan(chl_norm)
weighted_sum[mask] += alpha * chl_norm[mask]
weights_sum[mask] += alpha

# sst contribution
mask = ~np.isnan(sst_score)
weighted_sum[mask] += beta * sst_score[mask]
weights_sum[mask] += beta

# swh contribution
mask = ~np.isnan(swh_norm)
weighted_sum[mask] += gamma * swh_norm[mask]
weights_sum[mask] += gamma

# evitar divisão por zero
with np.errstate(invalid='ignore', divide='ignore'):
    indice = weighted_sum / weights_sum
indice[weights_sum == 0] = np.nan

# 8) Salvar resultado em NetCDF no Drive
print("Salvando índice combinado no Drive:", saida_nc)
ds_out = xr.Dataset(
    {
        "indice_tubarao": (("lat","lon"), indice)
    },
    coords = {
        "lat": lat_centers,
        "lon": lon_centers
    }
)
# incluir atributos explicativos
ds_out['indice_tubarao'].attrs['description'] = "Índice combinado de probabilidade de forrageamento de tubarões (0-1)."
ds_out['indice_tubarao'].attrs['weights'] = f"alpha={alpha},beta={beta},gamma={gamma}"
ds_out.attrs['notes'] = "Índice gerado a partir de PACE chlor_a, MODIS SST (regrid) e SWOT swh (rasterized)."

# Salvar NetCDF
try:
    ds_out.to_netcdf(saida_nc)
    print("Arquivo salvo com sucesso.")
except Exception as e:
    print("Erro ao salvar NetCDF:", e)

# 9) Mostrar resumo final
print("PRÉ-PROCESSAMENTO CONCLUÍDO. Resumo dos arrays:")
print("chl_norm:", np.nanmin(chl_norm), np.nanmax(chl_norm))
print("sst_score:", np.nanmin(sst_score), np.nanmax(sst_score))
print("swh_norm:", np.nanmin(swh_norm), np.nanmax(swh_norm))
print("indice:", np.nanmin(indice), np.nanmax(indice))

# (sugestão: se quiser melhorar a reamostragem espacial, instale e use 'xesmf' para remapeamento conservativo/interpolado)


In [ ]:
# === CÉLULA 7 – Visualização e salvamento do mapa final ===

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os

# Caminho do arquivo salvo anteriormente
output_file = "/content/drive/My Drive/projetos_tubaroes/indice_tubarao.nc"

# Carregar o dataset com o índice calculado
ds_indice = xr.open_dataset(output_file)
indice = ds_indice['indice_tubarao']
lats = ds_indice['lat']
lons = ds_indice['lon']

# --- Visualização ---
plt.figure(figsize=(12,6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines(resolution='110m', color='black', linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linestyle=':', alpha=0.5)
ax.add_feature(cfeature.LAND, facecolor='lightgray')

# Mostrar o mapa com colormap azul-verde-vermelho (baixo → alto)
mesh = ax.pcolormesh(lons, lats, indice, cmap='turbo', vmin=0, vmax=1, transform=ccrs.PlateCarree())
plt.colorbar(mesh, ax=ax, label='Probabilidade de presença de tubarões (0–1)', orientation='vertical')

plt.title("Mapa de Probabilidade de Aparecimento de Tubarões")
plt.tight_layout()
plt.show()

# --- Salvamento em PNG ---
output_png = "/content/drive/My Drive/projetos_tubaroes/mapa_tubarao.png"
plt.savefig(output_png, dpi=300, bbox_inches='tight')
print(f"✅ Mapa salvo com sucesso em: {output_png}")


In [ ]:
# === CÉLULA 8 – Mapa interativo com Plotly ===

import plotly.graph_objects as go
import numpy as np

# Carregar dados (mesmo que antes)
ds_indice = xr.open_dataset(output_file)
indice = ds_indice['indice_tubarao'].values
lats = ds_indice['lat'].values
lons = ds_indice['lon'].values

# Criar meshgrid para Plotly
lon2d, lat2d = np.meshgrid(lons, lats)

# Mapa interativo
fig = go.Figure(data=go.Heatmap(
    z=indice,
    x=lons,
    y=lats,
    colorscale='Turbo',  # azul -> verde -> vermelho
    zmin=0,
    zmax=1,
    colorbar=dict(title='Probabilidade de presença de tubarões (0–1)')
))

fig.update_layout(
    title='Mapa Interativo de Ocorrência de Tubarões',
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    yaxis=dict(scaleanchor="x"),  # manter proporção correta
    width=1000,
    height=500
)

fig.show()


In [ ]:
# ------------------ CÉLULA 9: MAPA INTERATIVO OTIMIZADO COM COR "TURBO" ------------------

import xarray as xr
import numpy as np
import folium
from folium import CircleMarker
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# --- Parâmetros ---
nc_path = '/content/drive/My Drive/projetos_tubaroes/indice_tubarao.nc'
map_save_path = '/content/drive/My Drive/projetos_tubaroes/mapa_interativo_tubarao.html'
threshold = 0.05  # filtra pontos muito baixos para não sobrecarregar o mapa
max_points = 50000  # limite de pontos para performance

# --- Carregar dados ---
ds = xr.open_dataset(nc_path)
indice_tubarao = ds['indice_tubarao']  # variável correta

# --- Selecionar pontos acima do limiar ---
lats_idx, lons_idx = np.where(indice_tubarao.values > threshold)
valores = indice_tubarao.values[lats_idx, lons_idx]

# --- Reduzir quantidade de pontos se necessário ---
if len(valores) > max_points:
    np.random.seed(42)
    selected_idx = np.random.choice(len(valores), max_points, replace=False)
    lats_idx = lats_idx[selected_idx]
    lons_idx = lons_idx[selected_idx]
    valores = valores[selected_idx]

# --- Criar mapa centralizado no globo ---
m = folium.Map(location=[0, 0], zoom_start=2, tiles='CartoDB positron')

# --- Função para converter valor em cor usando Turbo ---
turbo_cmap = cm.get_cmap('turbo')  # azul -> vermelho
def valor_para_cor(val):
    rgba = turbo_cmap(val)  # retorna RGBA 0-1
    return f'#{int(rgba[0]*255):02x}{int(rgba[1]*255):02x}{int(rgba[2]*255):02x}'

# --- Adicionar pontos ao mapa ---
for lat_i, lon_i, val in zip(lats_idx, lons_idx, valores):
    CircleMarker(
        location=[float(indice_tubarao.lat.values[lat_i]), float(indice_tubarao.lon.values[lon_i])],
        radius=3,
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=valor_para_cor(val)
    ).add_to(m)

# --- Salvar mapa interativo ---
m.save(map_save_path)
print(f"Mapa interativo salvo em: {map_save_path}")
